<a href="https://colab.research.google.com/github/sk27110/AutoPriceForecast/blob/main/scraping_autoru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sys
import time
import datetime
import requests as r
import json
from bs4 import BeautifulSoup
from pandas import Series
import requests as r, json
import re


START_URL='https://auto.ru/moskva/cars/used/'

In [3]:
pages_url_list=[]

for i in range(1, 200):
    response = r.get( START_URL+'?page=%s'%i)
    page = BeautifulSoup(response.text, 'html.parser')
    for link in page.findAll('a',{"class": "Link ListingItemTitle__link"}):
        try:
            pages_url_list.append(link['href'])
        except:
            pass

In [ ]:
cars_list = []

for item in pages_url_list:
  response = r.get(item)
  response.encoding = 'utf8'
  page = BeautifulSoup(response.text, 'html.parser')
  price = page.find('span', 'OfferPriceCaption__price')
  data = page.find('ul', 'CardInfo__list-MZpc1')
  car_info = {}
  if data and price:
    price = price.string
    price_number = re.sub(r'\s+', '', price).replace('₽', '')
    car_info['price'] = price_number
    for row in data.find_all('li', class_='CardInfoRow'):
      key = row.find('div', class_='CardInfoRow__cell').text.strip()
      value = row.find_all('div', class_='CardInfoRow__cell')[1].text.strip()
      car_info[key] = re.sub(r'\s+', '', value)
    cars_list.append(car_info)


df = pd.DataFrame(cars_list)


In [ ]:
df.to_csv('cars_data.csv', index=False, encoding='utf-8-sig')